<a href="https://colab.research.google.com/github/Rawigits/HTML-Generator-from-Table/blob/main/Python_Generate_html_From_xlsx_Table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

สร้างโฟลเดอร์สำหรับอัปโหลด Excel / Creating excel upload folder
จากนั้นให้อัปโหลดไฟล์ใส่ไว้ในโฟลเดอร์ที่สร้างชื่อ "_EXCEL_GENERATE_HTML"

In [1]:
# @title
!pip install -q gwpy # add %%capture to remove pip install display

from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
import os
drive_folder_path = '/content/gdrive/MyDrive/_EXCEL_GENERATE_HTML'
if not os.path.exists(drive_folder_path):
    os.makedirs(drive_folder_path)
# Change the current working directory to the newly created folder
os.chdir(drive_folder_path)

import pandas as pd
from IPython.display import display, HTML
import ipywidgets as widgets
import sys
# Function to get a list of Excel files in the folder
def get_excel_files():
    filetype = ('.xlsx', '.gsheet')
    excel_files = [file for file in os.listdir() if file.endswith(filetype)]
    if (len(excel_files) == 0):
        print("ใส่ไฟล์ Excel ไว้ที่โฟลเดอร์ _EXCEL_GENERATE_HTML")
    return excel_files
# Get the list of Excel files
excel_files = get_excel_files()
# Display the list of Excel files
file_list_widget = widgets.Dropdown(
    options=excel_files,
    description='เลือก Excel File:'
)
display(file_list_widget)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/gdrive


Dropdown(description='เลือก Excel File:', options=('DOS ITEM Price List.xlsx', 'Shera Item not use.gsheet', 'S…

ป้อมลำดับหมายเลขชีทของ Excel

In [2]:
# @title
sheetnumber = input("Enter sheet number: ")

Enter sheet number: 3


โหลดไฟล์ YAML เวอร์ชั่นล่าสุด / Update lastest version YAML

โปรดใส่คำค้นหาคอลัมน์แรก

In [3]:
# @title
selected_file = file_list_widget.value
xlsx_path = selected_file

import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/Rawigits/HTML-Generator-from-Table/main/config.yaml", "config.yaml")
print("Selected File:", xlsx_path)
yaml_path = "config.yaml"
findtext = input("Enter First column: ")

Selected File: Shera Item.xlsx
Enter First column: ไม้ฝาสเปลนดิด


เปิดการทำงานอ่านค่าจาก Excel และสร้างตาราง html

In [7]:
# @title
%%capture
%pip install openpyxl
import os
import re
import openpyxl

def reader(filepath, sheetnumber, findtext):
    textlist = []
    workbook = openpyxl.load_workbook(filepath)
    sheets = workbook.sheetnames
    # sh = workbook[sheets[0]]
    sh = workbook[sheets[int(sheetnumber) - 1]]

    for row in range(1, sh.max_row + 1):
        if str(findtext) in str(sh.cell(row,1).value) :
            for selected_row in range(row, sh.max_row + 1):
                row_text = []
                for column in range(2, sh.max_column + 2):
                    if sh.cell(selected_row,column).value != None:
                        row_text += [sh.cell(selected_row,column).value]
                    else:
                        textlist.append(row_text)
                        # print (textlist)
                        break
                if sh.cell(selected_row + 1,1).value != None:
                    break
        else:
            pass
    return textlist
textlist = reader(xlsx_path, sheetnumber, findtext)


%pip install ruamel.yaml
from ruamel.yaml import YAML

def open_configfile(action, index , newvalue = None):
    if (action == "getvalue"):
        with open("config.yaml", "r") as yamlfile:
            yaml=YAML(typ="rt")
            yaml.preserve_quotes = True
            yaml.default_flow_style = False
            yaml.default_style = '|'

            configfile = yaml.load(yamlfile)
            value = configfile[index]
            yamlfile.close()
            return value

    elif (action == "setvalue"):
        with open("config.yaml", "r") as yamlfile:
            yaml=YAML()
            configfile = yaml.load(yamlfile)

        configfile[index] = str(newvalue)
        with open("config.yaml", "w", encoding="utf-8") as yamlfile:
            yaml.preserve_quotes = True
            yaml.default_flow_style = False
            yaml.default_style = '|'

            yaml.dump(configfile, yamlfile)
            print(f"Saved {newvalue} to {index}")
            yamlfile.close()
    else:
        print("wrong action")

def generatehtml(textlist):
    tablestyle = open_configfile(action="getvalue", index="style_green_header_table")
    tableclass = open_configfile(action="getvalue", index="tableclass")
    # print(f'{tablestyle}\n{tableclass}')

    if (textlist == None):
        print("No textlist")
        return

    tableend = '</table>'
    htmltext = ''
    texthead = '  <tr>'
    textrow = ''

    for i,textget in enumerate(textlist):
        if (i == 0):
            headercount = 0
            for text in textget:
                texthead += f'''
    <th onclick="sortTable({headercount})" style="background-color:#00b312">{text}<span class="icon"></span></th>'''
                headercount += 1
            texthead += '''
  </tr>'''
        else:
            textrow += '\n  <tr>'
            for text in textget:
                textrow += f'''
    <td>{text}</td>'''
            textrow += '''
  </tr>'''
    # print('table html =\n',texthead)
    # print('row html =\n',textrow)
    htmltext = f'{tablestyle}\n{tableclass}\n{texthead}{textrow}\n{tableend}'
    return htmltext

html = generatehtml(textlist)

กดเพือแสดงผลลัพธ์

In [8]:
# @title
print(html)

<style>
.customtable {
  max-width: 100%;
  margin-top: 10px;
  margin-bottom: 10px;
  border-width: 1px;
  border: 1px solid black;
  border-collapse: collapse;
  border-style: solid;
  overflow-x: auto;
  margin: 0 auto;
}

th, td {
  color: #6d6d6f;
  font-family: 'Helvetica', 'Arial', sans-serif;
  font-size: 18px;
  font-weight: bold;
  border-width: 1px;
  border: 1px solid black;
  padding: 4px;
  text-align: center;
  cursor: pointer;
}

th {
  color: #ffffff;
  background-color: #00b312;
}

@media only screen and (max-width: 480px) {
  .customtable {
    width: 100%;
    margin: 0;
    overflow-x: auto;
  }

  th, td {
    font-size: 13px;
    padding: 2px;
  }
}

@media only screen and (max-width: 768px) and (min-width: 481px){
  .customtable {
    width: 100%;
    margin: 0;
    overflow-x: auto;
  }

  th, td {
    font-size: 16px;
    padding: 2px;
  }
}
</style>

<script>
  function sortTable(n) {
    var table, rows, switching, i, x, y, shouldSwitch, dir, switchcount = 0